In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipython_memory_usage
%ipython_memory_usage_start

'memory profile enabled'

In [1] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 99.60 MiB


In [2]:
%load_ext watermark
%watermark -i -v -m -p pandas,numpy,matplotlib -g -b

2020-06-12T19:17:47+01:00

CPython 3.8.3
IPython 7.15.0

pandas 1.0.4
numpy 1.18.5
matplotlib 3.2.1

compiler   : GCC 7.5.0
system     : Linux
release    : 5.6.7-050607-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git hash   : 4c1d4ac8e448a7cc4e47a21820afc56622346ea2
Git branch : master
In [2] used 0.7969 MiB RAM in 0.16s, peaked 0.00 MiB above current, total RAM usage 100.39 MiB


In [3]:
NROWS = 10_000_000
NBR_LOOPS = 100
MATH_FN = 'mean'

dtypes = ['int64', 'int32', 'int16', 'int8', 'uint8', 'float128', 'float64', 'float32', 'float16']
cols = {}
for dtyp in dtypes:
    # makes random data using normal distribution, then cast
    # to various dtypes (which will involve rounding)
    arr = np.random.normal(size=NROWS).astype(dtyp)
    cols[dtyp] = arr
df_data = pd.DataFrame(cols)
df_data.head(5)

,int64,int32,int16,int8,uint8,float128,float64,float32,float16
0,0,0,0,0,0,-1.580440,-0.090706,-0.050156,-0.040985
1,0,0,0,0,0,0.744341,-2.281576,-0.639802,0.694824
2,0,0,0,0,255,0.910785,0.869701,-1.025153,1.440430
3,0,0,0,-1,0,-0.715027,0.144067,-1.574106,1.317383
4,0,0,-1,0,0,-1.616509,0.942782,-0.335202,-0.783691


In [3] used 878.6016 MiB RAM in 7.93s, peaked 0.00 MiB above current, total RAM usage 979.00 MiB


In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 9 columns):
 #   Column    Dtype   
---  ------    -----   
 0   int64     int64   
 1   int32     int32   
 2   int16     int16   
 3   int8      int8    
 4   uint8     uint8   
 5   float128  float128
 6   float64   float64 
 7   float32   float32 
 8   float16   float16 
dtypes: float128(1), float16(1), float32(1), float64(1), int16(1), int32(1), int64(1), int8(1), uint8(1)
memory usage: 438.7 MB
In [4] used 0.1367 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 979.13 MiB


In [5]:
cols = []
for col in df_data.columns:
    print(f"Working on {col}")
    arr = df_data[col]
    for fn_idx in range(2):
        if fn_idx == 0:
            fn_name = "ser"
            fn = getattr(arr, MATH_FN)
        else:
            fn = getattr(getattr(arr, 'values'), MATH_FN)
            fn_name = "ser.values"
        for n in range(NBR_LOOPS):
            t1 = time.time()
            #arr.mean()
            fn()
            #timings[n] = time.time() - t1
            delta = time.time() - t1
            cols.append({'fn_name': fn_name, 'col': col, 't': delta})
df_results = pd.DataFrame(cols)
df_results.sample(5)

Working on int64
Working on int32
Working on int16
Working on int8


KeyboardInterrupt: 

In [5] used -437.5352 MiB RAM in 7.95s, peaked 437.54 MiB above current, total RAM usage 541.60 MiB


In [ ]:
df_results.info()

In [ ]:
gpby = df_results.groupby(['fn_name', 'col'])
means = gpby.mean().unstack(0)
means = means.loc[dtypes]
means

In [ ]:
stds = gpby.std().unstack(0)
stds = stds.loc[dtypes]
se = stds / np.sqrt(NBR_LOOPS)
se_95pc = se * 1.96
se_95pc

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
means.plot(kind='bar', ax=ax, yerr=se_95pc)

title = f"Fn '{MATH_FN}' execution time on {NROWS:,} rows of normal rnd over {NBR_LOOPS:,} loops"
title += f"\nBlack bars are the 95% Confidence Interval"
title += "\nUsing Pandas Series and Series.values"
ax.set_title(title);
ax.set_ylabel('Seconds (smaller is better)');

locs, labels=plt.yticks()
y_ticks = []
new_yticks=[f"{d}s" for d in locs]
plt.yticks(locs,new_yticks); #, rotation=45, horizontalalignment='right')
ax.set_xlabel('dtype');
ax.get_figure().savefig('timings.png')
plt.show()